In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import requests
import matplotlib.pyplot as plt
import networkx as nx

In [ ]:
#Визуализация
!pip install keplergl
from keplergl import KeplerGl
from google.colab import output
output.enable_custom_widget_manager()

## Подготовка данных
Для расчетов нам потребуются слои жилых домов, сервисов (в данном случае школ) и дорожный граф

In [177]:
schools = gpd.read_file('./input_data/school.geojson')
houses = gpd.read_file('./input_data/Buildings_output.geojson')
road_graph = nx.read_graphml('./input_data/walk_graph.graphml', node_type=int)

In [169]:
houses.head()

,id,is_living,building,building:levels,area_residential,people,area_total,geometry
0,33596,True,hut,3,45,2,135,"POLYGON ((344784.461 6309898.149, 344782.519 6..."
1,33595,True,hut,3,29,1,87,"POLYGON ((344716.830 6309844.191, 344714.601 6..."
2,33800,True,apartments,2,234,7,292,"POLYGON ((349802.200 6295991.096, 349793.574 6..."
3,33806,True,yes,1,48,1,60,"POLYGON ((353187.307 6298671.303, 353189.018 6..."
4,33577,True,hut,3,30,1,90,"POLYGON ((344749.552 6309835.795, 344748.337 6..."


In [170]:
schools.head()

,Наименование,Широта,Долгота,Вместимость,geometry
0,Во имя святых Царственных страстотерпцев,56.741941,60.609701,259,POINT (353807.724 6291211.181)
1,Гимназия им. Святейшего Патриарха Алексия Второго,56.809546,60.697879,940,POINT (359452.559 6298549.225)
2,Гимназия № 70,56.811099,60.596437,940,POINT (353266.890 6298934.933)
3,Гимназия №108 им. В.Н. Татищева,56.850533,60.649841,830,POINT (356677.110 6303209.717)
4,Гимназия №116,56.819829,60.562072,1 860,POINT (351204.086 6299980.538)


In [178]:
#Фильтрация датасетов - отбираем только жилые здания, приводим формат вместимости школ к целочисленному, устанавливаем стандартную проекцию
houses = houses[houses['is_living'] == 1.0]
houses.reset_index(inplace=True, drop=True)
schools['Вместимость'] = schools.apply(lambda x : "".join(x['Вместимость'].split()), axis=1).astype(int)
schools = schools.to_crs(32641)
houses = houses.to_crs(32641)

In [167]:
houses.head()

,id,is_living,building,building:levels,area_residential,people,area_total,geometry
0,33596,True,hut,3,45,2,135,"POLYGON ((344784.461 6309898.149, 344782.519 6..."
1,33595,True,hut,3,29,1,87,"POLYGON ((344716.830 6309844.191, 344714.601 6..."
2,33800,True,apartments,2,234,7,292,"POLYGON ((349802.200 6295991.096, 349793.574 6..."
3,33806,True,yes,1,48,1,60,"POLYGON ((353187.307 6298671.303, 353189.018 6..."
4,33577,True,hut,3,30,1,90,"POLYGON ((344749.552 6309835.795, 344748.337 6..."


In [171]:
#Статистика по наслению в жилых домах
houses.people.describe()

count    25127.000000
mean        59.672504
std        146.549988
min          0.000000
25%          3.000000
50%          5.000000
75%         50.000000
max       3212.000000
Name: people, dtype: float64

## Расчет матрицы расстояний

In [ ]:
!pip install networkit

import numpy as np
import shapely
import geopandas as gpd
import shapely
import copy
import shapely.wkt
import networkit as nk
from tqdm.notebook import tqdm
tqdm.pandas()

from scipy import spatial
from shapely.geometry import LineString, Point, Polygon


"""Functions to calculate distance matrix with road network"""

def calculate_distance_matrix(road_network, houses, facilities, crs=32636, type=['walk'], weight='length_meter'):

    network = road_network.edge_subgraph(
    [(u, v, k) for u, v, k, d in road_network.edges(data=True, keys=True)
    if d["type"] in type]
    )

    # find nearest points to objects on road network
    gdf = gpd.GeoDataFrame.from_dict(dict(network.nodes(data=True)), orient='index')
    gdf["geometry"] = gdf.apply(lambda row: shapely.geometry.Point(row.x, row.y), axis=1)
    nodes_gdf = gpd.GeoDataFrame(gdf, geometry = gdf['geometry'], crs = crs)
    from_houses = nodes_gdf['geometry'].sindex.nearest(houses['geometry'], return_distance = True, return_all = False)
    to_facilities = nodes_gdf['geometry'].sindex.nearest(facilities['geometry'], return_distance = True, return_all = False)

    distance_matrix = pd.DataFrame(0, index = to_facilities[0][1], columns = from_houses[0][1])
    splited_matrix = np.array_split(distance_matrix.copy(deep = True), int(len(distance_matrix) / 1000) + 1)

    # conver nx graph to nk graph in oder to speed up the calculation
    nk_idmap = _get_nx2nk_idmap(network)
    net_nk =  _convert_nx2nk(network, idmap=nk_idmap, weight=weight)

    # calculate distance matrix
    for i in range(len(splited_matrix)):
        r = nk.distance.SPSP(G=net_nk, sources=splited_matrix[i].index.values).run()
        splited_matrix[i] = splited_matrix[i].apply(lambda x: _get_nk_distances(r,x), axis =1)
        del r

    distance_matrix = pd.concat(splited_matrix)
    distance_matrix.index = list(facilities.iloc[to_facilities[0][0]].index)
    distance_matrix.columns = list(houses.iloc[from_houses[0][0]].index)

    del splited_matrix

    # replace 0 values (caused by road network sparsity) to euclidian distance between two points
    distance_matrix = distance_matrix.progress_apply(lambda x: _calculate_euclidian_distance(x, houses, facilities))
    return distance_matrix


def _calculate_euclidian_distance(loc, houses, facilities):
    s = copy.deepcopy(loc)
    s_0 = s[s == 0]
    if len(s_0) > 0:
        s.loc[s_0.index] = facilities["geometry"][s.index].distance(houses["geometry"][s.name])
        return s
    else:
        return s


"""Functions to convert Networkx graph to Networkit graph"""

def _get_nx2nk_idmap(G_nx):
    idmap = dict((id, u) for (id, u) in zip(G_nx.nodes(), range(G_nx.number_of_nodes())))
    return idmap

def _convert_nx2nk(G_nx, idmap=None, weight=None):

    if not idmap:
        idmap = _get_nx2nk_idmap(G_nx)
    n = max(idmap.values()) + 1
    edges = list(G_nx.edges())

    if weight:
        G_nk = nk.Graph(n, directed=G_nx.is_directed(), weighted=True)
        for u_, v_ in edges:
                u, v = idmap[u_], idmap[v_]
                d = dict(G_nx[u_][v_])
                if len(d) > 1:
                    for d_ in d.values():
                            v__ = G_nk.addNodes(2)
                            u__ = v__ - 1
                            w = round(d_[weight], 1) if weight in d_ else 1
                            G_nk.addEdge(u, v, w)
                            G_nk.addEdge(u_, u__, 0)
                            G_nk.addEdge(v_, v__, 0)
                else:
                    d_ = list(d.values())[0]
                    w = round(d_[weight], 1) if weight in d_ else 1
                    G_nk.addEdge(u, v, w)
    else:
        G_nk = nk.Graph(n, directed=G_nx.is_directed())
        for u_, v_ in edges:
                u, v = idmap[u_], idmap[v_]
                G_nk.addEdge(u, v)

    return G_nk

def _get_nk_distances(nk_dists, loc):
    target_nodes = loc.index
    source_node = loc.name
    distances = [nk_dists.getDistance(source_node, node) for node in target_nodes]
    return pd.Series(data = distances, index = target_nodes)

In [172]:
distance_matrix = calculate_distance_matrix(road_network=road_graph, houses=houses, facilities=schools, crs=32641)

<ipython-input-3-a5d49d931842>:28: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf["geometry"] = gdf.apply(lambda row: shapely.geometry.Point(row.x, row.y), axis=1)


  0%|          | 0/25127 [00:00<?, ?it/s]

In [174]:
distance_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,25117,25118,25119,25120,25121,25122,25123,25124,25125,25126
0,24646.4,24744.8,8585.6,8544.0,24577.0,24577.0,24577.0,24471.7,24577.0,24577.0,...,22126.9,21980.1,21980.1,21980.1,22286.0,21897.0,22196.0,13586.6,13586.6,8813.6
1,21709.2,21807.6,12139.9,8046.2,21639.8,21639.8,21639.8,21534.5,21639.8,21639.8,...,13746.8,13600.0,13600.0,13600.0,13905.9,13516.9,13815.9,15849.0,15849.0,8684.0
2,16394.4,16492.8,5524.7,333.4,16325.0,16325.0,16325.0,16219.7,16325.0,16325.0,...,15684.9,15538.1,15538.1,15538.1,15844.0,15455.0,15754.0,8999.9,8999.9,1598.0
3,16583.3,16681.7,12031.2,7553.9,16513.9,16513.9,16513.9,16408.6,16513.9,16513.9,...,8965.9,8819.1,8819.1,8819.1,9125.0,8736.0,9035.0,13171.7,13171.7,8214.2
4,14462.0,14560.4,5161.9,2897.5,14392.6,14392.6,14392.6,14287.3,14392.6,14392.6,...,15839.7,15692.9,15692.9,15692.9,15998.8,15609.8,15908.8,6453.3,6453.3,2177.1


In [179]:
#Сохраняем матрицу
distance_matrix.to_csv('./output_data/distance_matrix.csv', index=False)

## Получение матрицы вероятностей

In [194]:
distance_matrix.fillna(0, inplace=True)
distance_matrix = distance_matrix.apply(lambda rowsum: 1 / rowsum if rowsum.sum() > 0 else rowsum, axis = 1)
distance_matrix.replace(np.inf, np.nan)

probabilities = distance_matrix.fillna(0)
sum_houses_distances = probabilities.sum(axis=0)

probabilities_matrix = probabilities.divide(sum_houses_distances, axis=1)

probabilities_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,25117,25118,25119,25120,25121,25122,25123,25124,25125,25126
0,0.002878,0.002890,0.003744,0.002353,0.002870,0.002870,0.002870,0.002857,0.002870,0.002870,...,0.002786,0.002794,0.002794,0.002794,0.002805,0.002799,0.002794,0.003589,0.003589,0.002716
1,0.003267,0.003279,0.002648,0.002498,0.003259,0.003259,0.003259,0.003247,0.003259,0.003259,...,0.004484,0.004516,0.004516,0.004516,0.004496,0.004534,0.004489,0.003077,0.003077,0.002757
2,0.004327,0.004335,0.005818,0.060288,0.004320,0.004320,0.004320,0.004310,0.004320,0.004320,...,0.003930,0.003953,0.003953,0.003953,0.003946,0.003966,0.003937,0.005418,0.005418,0.014981
3,0.004277,0.004286,0.002672,0.002661,0.004271,0.004271,0.004271,0.004261,0.004271,0.004271,...,0.006875,0.006965,0.006965,0.006965,0.006851,0.007016,0.006865,0.003702,0.003702,0.002914
4,0.004905,0.004911,0.006227,0.006937,0.004900,0.004900,0.004900,0.004893,0.004900,0.004900,...,0.003891,0.003914,0.003914,0.003914,0.003907,0.003926,0.003899,0.007556,0.007556,0.010996


### Расчет спроса матрицы спроса
На основе заселенности домов и вероятности расчитываем и суммируем спрос, а также получаем показатель удовлетворения спроса каждой отдельной школы

In [195]:
demand_matrix = probabilities_matrix.apply(lambda col : col * int(houses.loc[int(col.name)]['people']), axis=0)

demand_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,25117,25118,25119,25120,25121,25122,25123,25124,25125,25126
0,0.005756,0.002890,0.026208,0.002353,0.002870,0.002870,0.002870,0.002857,0.002870,0.005739,...,0.011142,0.002794,0.008383,0.008383,0.008415,0.011196,0.005589,0.125609,0.050244,0.010865
1,0.006535,0.003279,0.018535,0.002498,0.003259,0.003259,0.003259,0.003247,0.003259,0.006518,...,0.017935,0.004516,0.013549,0.013549,0.013487,0.018137,0.008979,0.107679,0.043071,0.011027
2,0.008653,0.004335,0.040729,0.060288,0.004320,0.004320,0.004320,0.004310,0.004320,0.008641,...,0.015719,0.003953,0.011859,0.011859,0.011837,0.015862,0.007874,0.189624,0.075850,0.059924
3,0.008555,0.004286,0.018703,0.002661,0.004271,0.004271,0.004271,0.004261,0.004271,0.008542,...,0.027498,0.006965,0.020894,0.020894,0.020553,0.028062,0.013730,0.129565,0.051826,0.011658
4,0.009810,0.004911,0.043591,0.006937,0.004900,0.004900,0.004900,0.004893,0.004900,0.009801,...,0.015565,0.003914,0.011742,0.011742,0.011722,0.015705,0.007798,0.264453,0.105781,0.043985


In [204]:
#В качестве правдоподобности берем только школьников, которых в Екатеринбурге приблизительно 10%
schools['total_demand'] = demand_matrix.sum(axis=1) * 0.1
schools['meeting_demand'] = schools['Вместимость'] / schools['total_demand']

schools.head()

,Наименование,Широта,Долгота,Вместимость,geometry,total_demand,meeting demand,meeting_demand
0,Во имя святых Царственных страстотерпцев,56.741941,60.609701,259,POINT (353807.724 6291211.181),491.815188,0.526621,0.526621
1,Гимназия им. Святейшего Патриарха Алексия Второго,56.809546,60.697879,940,POINT (359452.559 6298549.225),528.379847,1.779023,1.779023
2,Гимназия № 70,56.811099,60.596437,940,POINT (353266.890 6298934.933),896.025789,1.049077,1.049077
3,Гимназия №108 им. В.Н. Татищева,56.850533,60.649841,830,POINT (356677.110 6303209.717),800.416596,1.036960,1.036960
4,Гимназия №116,56.819829,60.562072,1860,POINT (351204.086 6299980.538),828.757979,2.244322,2.244322


### Расчет доступности сервиса для каждого жилого дома
Теперь вводим ограничение в виде расстояния до сервиса длиной в 1 км. Для каждого дома находим набор школ в данной зоне доступности и суммируем показатель удовлетворения спроса

In [198]:
#Возвращаем изначальную матрицу расстояний
distance_matrix = pd.read_csv('./output_data/distance_matrix.csv')

In [205]:
#Радиус зоны доступности
distance = 1000

#Расчитываем суммируемый показатель удовлетворения спроса для каждого дома и сохраняем в новый столбец
service_provision_column = []

for house_id,_ in houses.iterrows():
  str_id = str(house_id)
  schools_indexes = distance_matrix.loc[distance_matrix[str_id] <= distance].index
  house_provision = schools.loc[schools_indexes].meeting_demand.sum()
  service_provision_column.append(house_provision)

houses['service_provision'] = pd.Series(service_provision_column, index=houses.index)

houses.head()

,id,is_living,building,building:levels,area_residential,people,area_total,geometry,service_provision
0,33596,True,hut,3,45,2,135,"POLYGON ((344784.461 6309898.149, 344782.519 6...",0.000000
1,33595,True,hut,3,29,1,87,"POLYGON ((344716.830 6309844.191, 344714.601 6...",0.000000
2,33800,True,apartments,2,234,7,292,"POLYGON ((349802.200 6295991.096, 349793.574 6...",3.507423
3,33806,True,yes,1,48,1,60,"POLYGON ((353187.307 6298671.303, 353189.018 6...",4.271852
4,33577,True,hut,3,30,1,90,"POLYGON ((344749.552 6309835.795, 344748.337 6...",0.000000


In [206]:
#Статистика по доступности школы в радиусе 1 км
houses.service_provision.describe()

count    25127.000000
mean         2.440623
std          2.329789
min          0.000000
25%          0.000000
50%          2.047542
75%          3.961517
max         14.163959
Name: service_provision, dtype: float64

In [208]:
#Сохраняем полученный результат
houses.to_file('./output_data/houses_provision.geojson', driver='GeoJSON')
schools.to_file('./output_data/schools_provision.geojson', driver='GeoJSON')